# Hikurangi-pyvolumeDemo

Use the **pyvolume** library to produce an interactive 3D visualisation of Hikurangi.

In [12]:
import ipyvolume as ipv
# import ipyvolume
import requests
import numpy as np

In [2]:
!pip install geopandas >/dev/null
!pip install nest_asyncio > /dev/null
import geopandas as gpd

In [3]:
#for data animations
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [4]:
# tiles
url30 = "https://github.com/GNS-Science/eq-fault-geom/blob/chris_ruptures_101/data/subduction/tile_outlines_30.zip?raw=true"
url10 = "https://github.com/GNS-Science/eq-fault-geom/blob/chris_ruptures_101/data/subduction/tile_outlines.zip?raw=true"

tile_outlines_nztm = gpd.read_file(url10).to_crs(epsg=2193)

In [5]:
def quadPolyToTriPoly(coords, idx):
    assert len(coords) == 5
    x = [g[0] for g in coords[:-1]]
    y = [g[1] for g in coords[:-1]]
    z = [g[2] for g in coords[:-1]]
    faces = [[idx, idx+1, idx+2], [idx, idx+2, idx+3]]
    return x, y, z, faces
    

In [6]:
# build the fault section tiles 
idx = 0
x,y,z,t = [], [], [], []  
for bdry in tile_outlines_nztm.geometry.boundary:
    x0,y0,z0,t0 = quadPolyToTriPoly(list(bdry.coords), idx)
    x.extend(x0)
    y.extend(y0)
    z.extend(z0)
    t.extend(t0)
    idx +=4

In [7]:
#build the rupture face data
idx = 0
x1,y1,z1,t1 = [], [], [], []  
for bdry in tile_outlines_nztm.geometry.boundary[:540]:
    x0,y0,z0,t0 = quadPolyToTriPoly(list(bdry.coords), idx)
    x1.extend(x0)
    y1.extend(y0)
    z1.extend(z0)
    t1.extend(t0)
    idx +=4
    

In [8]:
#draw the tetrahedron
fig = ipv.figure()
mesh = ipv.plot_trisurf(x=x, y=y, z=z, triangles=t, color='lightblue')
faces = ipv.plot_trisurf(x1, y1, z1, triangles=t1, color='red')
ipv.ylim( min(y), max(y))
ipv.xlim( min(x), max(x))
ipv.zlim(-250e3, 0)
ipv.show()

In [14]:
# use async co-routines to update data 
async def new_faces(n):
    faces.triangles = np.array(t1[:n], dtype='uint32')
    await asyncio.sleep(0.2)

for n in range(-2, -1 * len(t1), -20):
    task = asyncio.create_task(new_faces(n))
    await task
